In [1]:
import numpy as np
import pandas as pd
import json

In [ ]:
#Get sample DOIs list
#Skip if you have your list of DOIs
from crossref.restful import Works

works = Works()
dois=[]
for item in works.sample(100):
   dois.append(item['DOI'])

with open("files/dois.txt", "w") as f:
    for doi in dois:
        f.write(str(doi) +"\n")   
    f.close()

In [2]:
#Load DOIs
file = open('files/dois.txt','r')
dois = file.read().split('\n')
file.close()

## Get Papers Info

In [5]:
import data_extraction

In [3]:
# Get references
def get_references(dois):
    papers={}
    for doi in dois:
        work=data_extraction.get_data(doi)
        if work:
            ref=data_extraction.references(work,doi)
            title,year=data_extraction.title_year(work)
            papers[doi]={'title':title,'ref':ref,'year':year}
    return papers

In [6]:
papers=get_references(dois)

Error fetching title for 10.1371/journal.pone.0323002.s004: list index out of range
Error fetching title for 10.1371/journal.pone.0237584.g003: list index out of range
Error fetching title for 10.1371/journal.pone.0218687.s012: list index out of range
Error fetching title for 10.22206/cys.2007.v32i1: list index out of range
Error fetching title for 10.1117/12.3055189.53cd772d-7d8f-ef11-a9a2-00505691c5e1: list index out of range
Error fetching title for 10.1111/jpe.2006.43.issue-5: list index out of range
Error fetching title for 10.1371/journal.pgen.1004581.g004: list index out of range
Error fetching title for 10.1093/oed/5737451289: list index out of range


In [7]:
with open('files/papers_info.json', 'w') as fp:
    json.dump(papers, fp)

### Get References Count

In [5]:

with open('files/papers_info.json', 'r') as fp:
    papers = json.load(fp)


In [8]:
def get_ref_counts(data):
    references=[]
    for key in list(data.keys()):
        references.extend(data[key]['ref'])
    values, counts = np.unique(references, return_counts=True)
    ref_counts={}
    for i,value in enumerate(values):
        ref_counts[value]=counts[i]
    return references,ref_counts

In [9]:
references_all,references_unique=get_ref_counts(papers)
print(len(references_all),len(list(references_unique.keys())))

1350 1347


In [10]:
def get_most_cited(references,freq=10):
    most_cited={}
    for key in list(references.keys()):
        if references[key]>=freq:
            most_cited[key]=references[key]
    return most_cited

In [13]:
most_cited=get_most_cited(references_unique,1)
len(list(most_cited.keys()))

1347

# Most Cited Info

In [14]:
papers_info=[]
for doi in list(most_cited.keys()):
    if doi in list(papers.keys()):
        year=papers[doi]['year']
        title=papers[doi]['title']
        in_dataset='Yes'
    else:
        data=data_extraction.get_data(doi)
        in_dataset='No'
        if data:
            title,year=data_extraction.title_year(data)
        else:
            title,year='-','-'

    papers_info.append([title,doi,most_cited[doi], in_dataset, year])


Error fetching data for 10.1002/(SICI)1521-4141(200004)30:4<1127::AID-IMMU1127>3.0.CO;2-#: 404 Client Error: Not Found for url: https://api.crossref.org/works/10.1002/(SICI)1521-4141(200004)30:4%3C1127::AID-IMMU1127%3E3.0.CO;2-
Error fetching title for 10.1023/a:1007753421236: list index out of range


In [15]:
most_cited_df=pd.DataFrame(data=papers_info,columns=['title','doi','no_citations','in_dataset','year'])
most_cited_df=most_cited_df.sort_values(by=['no_citations'],ascending=False)
most_cited_df.to_csv('files/most_cited.csv')